In [1]:
import lettuce as lt
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import csv
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter


In [1]:
parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
parser.add_argument("--vtkdir", type=str, help="name base of vtk files - they are usually in a vtk-subdirectory!", default="./output/")
parser.add_argument("--csvdir", type=str, default="./output/")
parser.add_argument("--resolution", type=int, default=16)
parser.add_argument("--nout", type=int, default=10)
parser.add_argument("--nvtk", type=int, default=200)
parser.add_argument("--tmax", type=int, default=10)
parser.add_argument("--Re", type=int, default=100)
args, unknown = parser.parse_known_args()
args = vars(args)
#args = vars(parser.parse_args())
Re=args["Re"]
basedir = args["vtkdir"]
csvdir = args["csvdir"]
res = args["resolution"]
nout = args["nout"]
nvtk = args["nvtk"]
tmax = args["tmax"]
#print(args)

#print(unknown)

NameError: name 'ArgumentParser' is not defined

In [1]:
lattice = lt.Lattice(lt.D3Q19, device = "cuda", dtype=torch.float64)
flow = lt.SuperReducedTaylorGreenVortex3D(resolution=res, reynolds_number=Re, mach_number=0.05, lattice=lattice)
print(flow.units.convert_time_to_lu(tmax))
collision = lt.BGKCollision3D(lattice, tau=flow.units.relaxation_parameter_lu)
print(flow.units.relaxation_parameter_lu)
streaming = lt.StandardStreaming(lattice)
simulation = lt.SimulationReducedTGV(flow=flow, lattice=lattice, collision=collision, streaming=streaming)

NameError: name 'lt' is not defined

In [3]:
interval_vtk=int((flow.units.convert_time_to_lu(tmax))/100)
interval_out=int((flow.units.convert_time_to_lu(tmax))/200)
Energy = lt.IncompressibleKineticEnergy(lattice, flow)
Dissipation_TGV=lt.Dissipation_TGV(lattice, flow)
reporter = lt.ObservableReporter(Energy, interval=interval_out, out=None)
reporter2= lt.ObservableReporter(Dissipation_TGV, interval=interval_out, out=None)
simulation.reporters.append(reporter)
simulation.reporters.append(reporter2)
simulation.reporters.append(lt.VTKReporter(lattice, flow, interval=interval_vtk, filename_base=basedir+"out"))

steps     time     IncompressibleKineticEnergy
steps     time     Dissipation_TGV


In [4]:
simulation.initialize_f_neq()
steps = int(flow.units.convert_time_to_lu(tmax))
mlups = simulation.step(num_steps=steps)
print("Performance in MLUPS:", mlups)
#rho_start,u_start,f_start=simulation.startwerte()

Performance in MLUPS: 12.290149537758225
rho_start
tensor([[[[1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 1.0009],
          [1.0003, 1.0003, 1.0003,  ..., 1.0009, 1.0009, 

In [6]:
energy = np.array(simulation.reporters[0].out)
dissipation = np.array(simulation.reporters[1].out)

dissipationEkin=[energy[:,1],-1/(np.pi*2)**3*np.gradient(64*energy[:,2],energy[:,1])]
dissipationEns=[dissipation[:,1],1/(0.5*np.pi)**3*dissipation[:,3]]
dissipationSij=[dissipation[:,1],dissipation[:,2]]

with open(csvdir + 'DissipationEkin.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(dissipationEkin)
with open(csvdir + 'DissipationEnstrophy.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(dissipationEns)
with open(csvdir + 'DissipationSij.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(dissipationSij)
#save_directory = outputdir
#file_name = "FullTGV.pkl"
#file_path = os.path.join(save_directory, file_name)
#simulation.save_checkpoint(file_path)

#csv_file_path = 'Brachet/Brachet_1983_tgv3d_enstrophy_Re' + str(Re) + '.csv'

# Daten aus der CSV-Datei lesen
#csv_data = []
#FullTGV_DisEns=[]
#FullTGV_DisEkin=[]
#FullTGV_DisSij=[]
#with open(csv_file_path, 'r') as file:
#    csv_reader = csv.reader(file, delimiter=';')
#    for row in csv_reader:
#        csv_data.append(row)

#with open('FullTGV/DisEns-Rey' + str(Re) + '-Res' + str(res) + '-BGK.csv', 'r') as file:
 #   csv_reader = csv.reader(file, delimiter=';')
#    for row in csv_reader:
 #       FullTGV_DisEns.append(row)

#with open('FullTGV/DisEkin-Rey' + str(Re) + '-Res' + str(res) + '-BGK.csv', 'r') as file:
#    csv_reader = csv.reader(file, delimiter=';')
#    for row in csv_reader:
#        FullTGV_DisEkin.append(row)#
#
#with open('FullTGV/DisSij-Rey' + str(Re) + '-Res' + str(res) + '-BGK.csv', 'r') as file:
#    csv_reader = csv.reader(file, delimiter=';')
#    for row in csv_reader:
#        FullTGV_DisSij.append(row)

#csv_data = np.array(csv_data, dtype=float)
#FullTGV_DisEns= np.array(FullTGV_DisEns, dtype=float)
#FullTGV_DisEkin= np.array(FullTGV_DisEkin, dtype=float)
#FullTGV_DisSij= np.array(FullTGV_DisSij, dtype=float)


#plt.figure(1)
#plt.scatter(csv_data[:, 0], csv_data[:, 1], color='red', label='Brachet')
#plt.plot(energy[:,1],-1/(np.pi*2)**3*np.gradient(64*energy[:,2],energy[:,1]),color = 'blue', label='Reduced')
#plt.plot(FullTGV_DisEkin[:,0],FullTGV_DisEkin[:,1],color='green', label='Full')
#plt.xlim(0,10)
#plt.ylim(0,0.02)
#plt.legend()
#title = 'Dissipation trough kinetic energy'
#plt.title(title)
#plt.savefig(imdir + title)

#plt.figure(2)
#plt.scatter(csv_data[:, 0], csv_data[:, 1], color='red', label='Brachet')
#plt.plot(dissipation[5:,1],1/(0.5*np.pi)**3*dissipation[5:,3], color = 'blue', label='Reduced')
#plt.plot(FullTGV_DisEns[:,0],FullTGV_DisEns[:,1],color='green', label= 'Full')
#plt.xlim(0,10)
#plt.ylim(0,0.02)
#plt.legend()
#title='Dissipation trough enstrophy'
#plt.title(title)
#plt.savefig(imdir + title)

#plt.figure(3)
#plt.scatter(csv_data[:, 0], csv_data[:, 1], color='red', label='Brachet')
#plt.plot(dissipation[3:,1],dissipation[3:,2], color = 'blue',label='Reduced')
#plt.plot(FullTGV_DisSij[:,0],FullTGV_DisSij[:,1],color = 'green',label='Full')
#plt.legend()
#plt.xlim(0,10)
#plt.ylim(0,0.02)
#title='Dissipation trough shear moments'
#plt.title(title)
#plt.savefig(imdir + title)

#plt.show()
#plt.close("all")

NameError: name 'simulation' is not defined

In [ ]:
# u = flow.units.convert_velocity_to_pu(lattice.u(simulation.f)).numpy()
# u_norm = np.linalg.norm(u,axis=0)
# plt.imshow(u_norm)
# plt.show()
# plt.close()